### Práctica ETL de Javier Bartolome Rodriguez

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

##### 1: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

El persists guarda datos en la memoria

El cache guarda datos en cache

El collect es lo mismo que un take

##### 2: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

Primero lo que tenemos que realizar es la carga de las librerías pertinentes y necesarias para hacer la prácica, luego los 
contextos necesarios que necesitemos. Más tarde leemos los datos con la ruta, los introducimos en nuestro contexto con sc().

Cargamos librerias

Cargamos los contextos que vayamos a usar

Realizamos la lectura de los datos

Introducimos los datos en nuestro contexto

#### 3: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Que las variables no esten en el tipo correcto

La falta de datos

La aparicion de filas y columnas sin informacion

El distinto formato de la fecha

#### 4: Inicializar spark context y cargar los datos desde el fichero.

In [3]:
import datetime as dt
import re
from datetime import *
import os
import pandas as pd
from time import time

from pyspark import SparkContext
from pyspark.sql import *

ModuleNotFoundError: No module named 'pyspark'

In [4]:
sc = SparkContext()

NameError: name 'SparkContext' is not defined

In [5]:
sqlContext = SQLContext(sc)

NameError: name 'SQLContext' is not defined

In [1]:
datos = './partidosLigaNBA.csv'
data = sc.textFile(datos)

NameError: name 'sc' is not defined

In [6]:
data.take(5)

NameError: name 'data' is not defined

In [7]:
limpieza = data.map(lambda x:x.split(':'))
limpieza.take(10)

NameError: name 'raw_data' is not defined

In [11]:
header = limpieza.first()

In [25]:
limpieza = data.map(lambda x:x.split(':')).filter(lambda x:'Playoffs' not in x[0]).filter(lambda x: x != header)
limpieza.take(5)

NameError: name 'data' is not defined

##### 5: Media de la diferencia de puntos por año

In [24]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt

ModuleNotFoundError: No module named 'pyspark'

In [23]:


data_frame = limpieza.map(lambda x:Row(fecha = dt.datetime.strptime((x[0]),'%a, %b %d, %Y').strftime('%m/%d/%Y'),
                                         hora_partido = (x[1] +' : '+ x[2]),
                                         local_team = x[5],
                                         visitor_team = x[3],
                                         score_visitante=int(x[4]),
                                         score_local=int(x[6])))



NameError: name 'limpieza' is not defined

In [22]:
df = sqlContext.createDataFrame(data_frame)
df.registerTempTable('interactions')

NameError: name 'sqlContext' is not defined

In [21]:
df.show(5)

NameError: name 'df' is not defined

In [20]:
df = df.withColumn('abs', abs(df.puntos_local - df.puntos_visitante)).show(2)


NameError: name 'df' is not defined

Por RDD

In [19]:
mean_rdd = limpieza.map(lambda x: (x[0][len(x)-11:18], __builtin__.abs(int(x[4]) - int(x[6]))))

miTupla = (0, 0)
mean_rdd.aggregateByKey(miTupla, lambda a,b: (a[0] + b,    a[1] + 1),
                                 lambda a,b: (a[0] + b[0], a[1] + b[1]))\
                                .mapValues(lambda x: x[0]/x[1])\
                                .sortBy(lambda x: x[0], False)\
                                .collect()

NameError: name 'limpieza' is not defined

## 5: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

In [8]:
from operator import add

In [12]:
eqVisitantes = limpieza.map(lambda y: (y[3], 1))
eqLocales = limpieza.map(lambda y: (y[5], 1))

agrLocales = eqLocales.reduceByKey(add).sortBy(lambda x: x[1], False).collect()
agrVisitantes = eqVisitantes.reduceByKey(add).sortBy(lambda x: x[1], False).collect()

NameError: name 'limpieza' is not defined

In [10]:
agrLocales

NameError: name 'agrLocales' is not defined

In [11]:
agrVisitantes

NameError: name 'agrVisitantes' is not defined

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

In [13]:
cleveVisitante = limpieza.filter(lambda x:x[3] == 'Cleveland Cavaliers').filter(lambda x:'Jan' in x[0]).filter(lambda x: int(x[4])>int(x[6]))
cleveVisitante.count()

NameError: name 'limpieza' is not defined

In [14]:
cleveLocal = limpieza.filter(lambda x:x[5] == 'Cleveland Cavaliers').filter(lambda x:'Jan' in x[0]).filter(lambda x: int(x[4])<int(x[6]))
cleveLocal.count()

NameError: name 'limpieza' is not defined

In [15]:
# Sumar para tener el ejercicio completo, como podemos observar tenemos 41 + 42

cleveLocal.count() + cleveVistante.count()

NameError: name 'cleveLocal' is not defined

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

In [16]:
warrior_local = limpieza.filter(lambda x:x[3] == 'Golden State Warriors').filter(lambda x: int(x[4])>int(x[6])).map(lambda x: (x[3],1)).reduceByKey(add).sortByKey().take(1)

NameError: name 'limpieza' is not defined

In [17]:
Warrior_visitante = limpieza.filter(lambda x:x[5] == 'Golden State Warriors').filter(lambda x:int(x[6])>int(x[4])).map(lambda x : (x[5],1)).reduceByKey(add).sortByKey().take(1)

NameError: name 'limpieza' is not defined

In [18]:
warrior_local + Warrior_visitante

NameError: name 'warrior_local' is not defined